In [ ]:
#%pip install -r requirements.txt

In [2]:
import polars as pl
import df_gen as dg
df = dg.generate_dummy_order_data(100)

In [3]:
#here's what we need to do to generate a delta table
df.write_delta('./delta_warehouse/orders')

In [5]:
# heres what we have to do to generate an iceberg table
from pyiceberg.catalog.sql import SqlCatalog
warehouse_path = "./iceberg_warehouse"
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{warehouse_path}/icyhot.db",
        "warehouse": f"file://{warehouse_path}",
    },
)
df_ice = df.to_arrow()
catalog.create_namespace_if_not_exists("dummy_data")
ice_table = catalog.create_table("dummy_data.orders",schema=df_ice.schema)
ice_table.append(df_ice)
# need this to get the latest metadata file to point to
print(ice_table.metadata_location)

file://./iceberg_warehouse/dummy_data.db/orders/metadata/00001-1a7cde4c-b8af-4109-ad35-952f49dab7a9.metadata.json
